In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import utils as utils
from datetime import datetime as dt
from newspaper import Article
import os

In [4]:
url = 'https://www.investing.com/equities/trending-stocks'
BASE_URL = 'https://www.investing.com'
request = requests.get(url).text
raw_text = BeautifulSoup(request, 'html.parser')
trending_stocks = raw_text.find('div', {'id': 'trendingInnerContent'})
stock_elements = trending_stocks.find_all('td', class_='left bold plusIconTd elp')
MAX_NUM_PAGES = 20

In [10]:
def extract_stock_info(stock_elements):
    stock_dict = {'company': [], 'stock': [], 'link': []}
    for element in stock_elements:
        company_name = element.find('a')['title']
        company_name = company_name.replace('\xa0', ' ')
        stock_name = element.find('a').text
        link = element.find('a')['href']
        full_link = f'{BASE_URL}{link}'
        stock_dict['company'].append(company_name)
        stock_dict['stock'].append(stock_name)
        stock_dict['link'].append(full_link)
    df = pd.DataFrame(stock_dict)
    return stock_dict, df
stock_dict, df = extract_stock_info(stock_elements)
# save stock_dict to json
utils.save_dict_to_json(stock_dict, '../datasets/trending_stocks.json')

# Match tickers to the stocks
Find the tickers names from chat gpt and save as ticker.json

In [11]:
tickers_dict = utils.load_dict_from_json('../datasets/ticker.json')
df['ticker'] = list(tickers_dict.values())
df.to_csv('../datasets/stocks.csv', index=False)